# Quora Question Pairs

The objective of this competition is to decide if a pair of questions has the same meaning. The point is to help avoid question duplication on Quora. The evaluation method uses Log Loss, defined as

$$ {\rm Log\_Loss} = -\frac{1}{n} \sum_{i=1}^n \left(y_i \log \hat{y}_i + (1-y_i)\log( 1-\hat{y}_i) \right) $$

Here $y_i$ is the truth of whether pair number $i$ is a dupplicate (i.e., equal to either $0$ or $1$) and $\hat{y}_i$ is the predicted probability that the pair is a duplicate. A perfect score is zero.

## Setup
Import useful packages for data analysis and plotting.

In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(context = 'notebook', font_scale = 1.5, rc={'figure.figsize':(10, 6)})

from IPython.display import display #for displaying multiple outputs from a single cell

import gensim #for word2vec
import nltk #natural language toolkit

import xgboost as xgb

import sklearn
import sklearn.neural_network
import sklearn.model_selection
import sklearn.decomposition

/Users/stefan/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Import Data

Import the training data.

In [93]:
train_data = pd.read_csv('train.csv')

## Compute Features
We will feed some features to our statistical model. In the interest of time, only a few features are used. More features would definitely be better. The features we use are mostly very simple, as well. We compute the number of words in each question, the number of characters, and the number of words the questions have in common. Slightly more sophisticated are the word2vec features. We use a pretrained word2vec model to associate a vector to each question, which is just the sum of the vectors associated to each word in a question (after removing stopwords). Then the two question vectors are compared using a few different distance measures.

In [8]:
###Load pretrained word2vec Model
gnewsvectors = '~/Documents/Data Science/Data/GoogleNews-vectors-negative300.bin.gz'
word2vec = gensim.models.KeyedVectors.load_word2vec_format(gnewsvectors, binary=True)

In [6]:
#Compute a Word2Vec vector associated to a question.
def qVec(question):
    #tokenize and remove stopwords
    question = nltk.word_tokenize(str(question).lower().decode('utf-8'))
    question = [word for word in question if word.isalpha()]
    question = [word for word in question if word not in nltk.corpus.stopwords.words('english')]
    #make a vector out of a question (as sum of vectors of words in question)
    vec = []
    for word in question:
        try:
            vec.append(word2vec[word])
        except:
            continue
    vec = np.array(vec).sum(axis=0)
    if type(vec) == np.float64:
        vec = np.zeros(300) #return zero vector for those questions that are all stopwords
    return vec

#Construct Word2Vec vectors of questions in data and add them to the dataframe
def makeWord2Vecs(data):
    data['q1vec'] = data.question1.apply(lambda x: qVec(x))
    data['q2vec'] = data.question2.apply(lambda x: qVec(x))
    
#Compute the number of words two questions have in common
def numInCommon(question1, question2):
    #Tokenize questions and remove stopwords
    question1 = nltk.word_tokenize(str(question1).lower().decode('utf-8'))
    question1 = [word for word in question1 if word.isalpha()]
    question1 = [word for word in question1 if word not in nltk.corpus.stopwords.words('english')]  
    question2 = nltk.word_tokenize(str(question2).lower().decode('utf-8'))
    question2 = [word for word in question2 if word.isalpha()]
    question2 = [word for word in question2 if word not in nltk.corpus.stopwords.words('english')]
    
    if len(question1)==0 or len(question2) == 0:
        return 0
    
    common_words = [word for word in question1 if word in question2]
    
    return len(common_words)

In [16]:
#List of features for future reference
features = ['q1Len','q2Len','q1NumWords','q2NumWords','wordsInCommon','euclidean', 'cosine', 'cityblock']
#This function adds columns to the dataframe representing new features.
def computeFeatures(data):
    #####Simple Features
    #Length of the question (in characters)
    data['q1Len'] = data.question1.apply(lambda x: len(str(x)))
    data['q2Len'] = data.question2.apply(lambda x: len(str(x)))
    #Number of words in the question
    data['q1NumWords'] = data.question1.apply(lambda x: len(str(x).split()))
    data['q2NumWords'] = data.question2.apply(lambda x: len(str(x).split()))
    #Number of words in common
    data['wordsInCommon'] = data.apply(lambda x: numInCommon(x['question1'],x['question2']) ,axis=1)
    
    
    #####Word2Vec Distance Measures. Cut down on the number to save processing time.   
    #Euclidean distance between questions
    data['euclidean'] = data.apply(lambda x: scipy.spatial.distance.euclidean(x['q1vec'],x['q2vec']), axis=1)
    #Cosine distance between questions
    data['cosine'] = data.apply(lambda x: scipy.spatial.distance.cosine(x['q1vec'],x['q2vec']), axis=1)
    #City block distance between questions
    data['cityblock'] = data.apply(lambda x: scipy.spatial.distance.cityblock(x['q1vec'],x['q2vec']), axis=1)
    #Canberra distance between questions
    #data['canberra'] = data.apply(lambda x: scipy.spatial.distance.canberra(x['q1vec'],x['q2vec']), axis=1)
    #Chebyshev distance between questions
    #data['chebyshev'] = data.apply(lambda x: scipy.spatial.distance.chebyshev(x['q1vec'],x['q2vec']), axis=1)
    #Minkowski p=3 distance between questions
    #data['minkowski'] = data.apply(lambda x: scipy.spatial.distance.minkowski(x['q1vec'],x['q2vec'], 3), axis=1)
    #Jaccard distance between questions
    #data['jaccard'] = data.apply(lambda x: scipy.spatial.distance.jaccard(x['q1vec'],x['q2vec']), axis=1)

In [94]:
#Compute word2vecs for each question
makeWord2Vecs(train_data)

In [95]:
computeFeatures(train_data)
train_data=train_data.dropna() #I didn't bother tracking down the source of the NaN. This would likely be useful.
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1vec,q2vec,q1Len,q2Len,q1NumWords,q2NumWords,wordsInCommon,euclidean,cosine,cityblock
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[-0.820496, 0.078125, -0.170593, -0.282959, -0...","[-0.586121, 0.149902, -0.181152, -0.609131, -0...",66,57,14,12,6,3.708764,0.068972,50.923897
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[-0.296875, 0.263794, -0.0942383, -0.0251465, ...","[-0.575684, 0.543213, 0.283691, 0.909668, -0.7...",51,88,8,13,2,7.875035,0.512164,107.384239
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[0.358398, -0.0273132, 0.311768, 0.163818, -0....","[0.00634766, 0.0675049, -0.274902, -0.0244141,...",73,59,14,10,2,5.520098,0.222009,74.963737
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[-0.0881348, 0.178955, -0.612556, 0.518799, -0...","[0.473511, -0.123413, 0.105713, 0.441162, -0.3...",50,65,11,9,0,7.063795,0.650411,99.030556
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[-0.876587, 0.0933838, 0.897949, 0.224121, -0....","[-0.250488, 0.742676, 0.192139, 0.206299, -0.5...",76,39,13,7,2,11.753539,0.369993,158.978638


## Classification Model
Now we use the features we've created to predict whether we have duplicates. We will use a XGBoost to construct the model. One interesting twist is that the training data that was supplied has a much larger fraction of duplicates than the test data. The XGBoost classfier works better when the training data has the same fraction of positives as the positives. The fraction of positives in the test data is $p=0.17$, so we pad the negatives in the training data until the fraction of positives drops to $p$. This is done in a not-very-intelligent way, and there is a risk of overtraining, but doing this seems to improve the results. I'm not too worried about the details of this procedure, though, since it doesn't really have anything to do with the actual task of detecting duplicate questions.

In [192]:
X = train_data[features]
Y = train_data['is_duplicate']

#Oversample negative results to get a total fraction p of positives.
#The idea for this code snippet came from a comment on the message boards.
p=0.17
num_pos = len(X[Y==1])
num_neg = len(X[Y==0])
neg_results = X[Y==0]
pos_results = X[Y==1]
factor = num_pos/float(num_neg*p)-1
while factor > 1:
    neg_results = pd.concat([neg_results,X[Y==0]])
    factor -=1
neg_results = pd.concat([neg_results,X[Y==0][:int(factor*len(X[Y==0]))]])
X = pd.concat([pos_results,neg_results]).reset_index(drop=True)
Y = pd.concat([pd.Series(np.ones(num_pos)),pd.Series(np.zeros(len(neg_results)))]).reset_index(drop=True)   


X_train, X_cv, Y_train, Y_cv = sklearn.model_selection.train_test_split(X,Y,test_size=0.2)

dtrain = xgb.DMatrix(X_train,label = Y_train)
dcv = xgb.DMatrix(X_cv,label = Y_cv)

params = {'objective':'binary:logistic','eval_metric':'logloss','eta':0.2,'max_depth':4}
watchlist = [(dtrain, 'train'), (dcv, 'cv')]

bst = xgb.train(params, dtrain, 400, watchlist, verbose_eval=50)

[0]	train-logloss:0.593421	cv-logloss:0.593359
[50]	train-logloss:0.340231	cv-logloss:0.340193
[100]	train-logloss:0.335793	cv-logloss:0.336516
[150]	train-logloss:0.333456	cv-logloss:0.334822
[200]	train-logloss:0.331439	cv-logloss:0.333469
[250]	train-logloss:0.329867	cv-logloss:0.332531
[300]	train-logloss:0.328442	cv-logloss:0.331656
[350]	train-logloss:0.327298	cv-logloss:0.331025
[399]	train-logloss:0.325989	cv-logloss:0.33021


## Create Submission

Creating the submission is as simple as processing the test data like we did for the training data and running the XGBoost model on it. Because our data processing throws out some of the data (never got around to tracking down exactly what happened there), we will need to replace that data in the submission. Since we can't apply our model to it, we just guess that those entries are duplicates with probability $p=0.17$, since that is the fraction of duplicates in the test data.

In [96]:
test_data = pd.read_csv('test.csv')
all_ids = test_data['test_id'] #will need this list for later
test_data.describe()

,test_id
count,2.345796e+06
mean,1.172898e+06
std,6.771731e+05
min,0.000000e+00
25%,5.864488e+05
50%,1.172898e+06
75%,1.759346e+06
max,2.345795e+06


In [97]:
makeWord2Vecs(test_data)
computeFeatures(test_data)
test_data=test_data.dropna()
test_data.head()

,test_id,question1,question2,q1vec,q2vec,q1Len,q2Len,q1NumWords,q2NumWords,wordsInCommon,euclidean,cosine,cityblock
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,"[-0.114532, 0.275391, 0.631348, -0.134033, -0....","[0.0960693, 0.323242, 0.634277, 0.692871, -1.1...",57,68,11,14,3,8.468845,3.954517e-01,114.431046
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,"[0.0454102, 0.900635, 0.343994, 0.495117, -0.3...","[-0.325195, 0.468506, 0.366943, 0.397949, -0.5...",66,43,14,7,4,3.817149,1.089034e-01,53.327644
2,2,What but is the best way to send money from Ch...,What you send money to China?,"[-0.116577, 0.238281, 0.506485, 1.20508, -0.24...","[-0.172852, 0.0881348, 0.239807, 0.691406, -0....",60,29,14,6,3,4.425264,1.636005e-01,62.257957
3,3,Which food not emulsifiers?,What foods fibre?,"[-0.218994, 0.065918, -0.291016, 0.28125, -0.5...","[-0.152344, 0.205078, 0.201172, 0.490234, -0.1...",27,17,4,3,0,3.878658,3.196543e-01,54.527912
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,"[0.0681152, 0.0792236, -0.119995, 0.458008, -0...","[0.0681152, 0.0792236, -0.119995, 0.458008, -0...",32,30,4,6,2,0.000000,9.255779e-08,0.000000


In [190]:
X_test = test_data[features]

dtest = xgb.DMatrix(X_test)

probs = bst.predict(dtest)
ids = test_data['test_id']
preds = pd.DataFrame(columns = ['test_id','is_duplicate'])
preds['test_id'] = ids
preds['is_duplicate'] = probs
#Now add missing test values back in.
missing_ids = set(all_ids) - set(ids)
missing_preds = pd.DataFrame(columns = ['test_id','is_duplicate'])
missing_preds['test_id'] = list(missing_ids)
missing_preds['is_duplicate'] = p
preds = pd.concat([preds,missing_preds]).sort_values(by = 'test_id')
preds.reset_index(drop=True, inplace=True)
preds.head()

,test_id,is_duplicate
0,0,0.084937
1,1,0.331026
2,2,0.303677
3,3,0.002113
4,4,0.221190


In [191]:
#Save predictions to csv file for submission.
#preds.to_csv('xgb-resample.csv', index=False)